> [!WARNING]
> É recomendado criar um ambiente virtual de desenvolvimento em Python no diretório **[machine-learning](./machine-learning)**. – _Utilizamos a versão 3.8.10_:
>
> ```bash
> python -m venv venv
> ```
>
> **Tenha certeza de estar no diretório machine-learning**
>
> Para ativar o ambiente virtual em dispositivo Windows:
>
> ```ps1
> venv\Scripts\activate
> ```
>
> Para ativar o ambiente virtual em dispositivo Linux:
> 
> ```bash
> source venv/bin/activate
> ```
>
> Para desativar o ambiente virtual:
>
> ```bash
> deactivate
> ```
>
> > [!NOTE]
> > É possivel que você encontre algum problema ao ativar o ambiente devido a alguma configuração de segurança de seu sistema operacional. Leia atentamente as instruções que serão exibidas no terminal e **em último caso**, considere desativar temporariamente as políticas de segurança de execução de scripts para poder executar o comando.
> 
> Instale as dependências através do arquivo [requirements.txt](./requirements.txt)
> 
> ```bash
> pip install -r requirements.txt
> ```
>
> > [!NOTE]
> > Sempre confira se o arquivo requirements.txt está presente e sempre o atualize caso adicione novas dependências com o comando:
> >
> > ```bash
> > pip freeze > requirements.txt
> > ```

<br>

> [!NOTE]
> É possível qua a aplicação não reconheça os imports do Python devido a configurações do interpretador da linguagem do seu editor de código.
>
> Para solucionar esse problema, acesse as configurações (Settings) e as abra em formato JSON (Open Settings (JSON)) se houver a possibilidade.
>
> Adicione o seguinte script ajustando o caminho para o seu ambiente virtual:
>
> ```json
> {
>     // Outras configurações...
>    
>     // Para Windows
>     "python.pythonPath": "caminho/para/seu/venv/Scripts/python.exe",
>
>     // Para Linux
>     "python.pythonPath": "caminho/para/seu/venv/bin/python",
>
>     // Caminho para o diretório pai do seu ambiente virtual. No nosso caso "machine-learning"
>     "python.venvPath": "caminho/para/seu/venv"
> }
> ```
>
> **Lembre-se de remover os comentários do arquivo antes de salvar as alterações.**
>
> Salve e reinicie o Editor.
>
> Altere o interpretador Python para o seu ambiente virtual (no VS Code o atalho é `Ctrl` + `Shift` + `p`).

Link dos Datasets: https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews

In [1]:
import pandas as pd
import numpy as np
import chardet
import seaborn as sns
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


In [ ]:
df_books = pd.read_csv('C:\\Users\\Pc Gamer\\Downloads\\basepi\\books_data.csv')
df_books.head()

In [ ]:
df_books = df_books[[ 'Title', 'authors', 'image', 'categories']].copy()
df_books.rename(columns={
    'Title': 'title'
}, inplace=True)
df_books

In [ ]:
df_ratings = pd.read_csv('C:\\Users\\Pc Gamer\\Downloads\\basepi\\Books_rating.csv')
df_ratings.head()

In [ ]:
df_ratings = df_ratings[[ 'Title', 'User_id', 'review/score']].copy()
df_ratings.rename(columns={
    'Title': 'title',
    'User_id' : 'user_id',
    'review/score' : 'rating'
    
}, inplace=True)
df_ratings.dropna(inplace=True)
df_ratings


In [ ]:
df_ratings.describe()

In [ ]:
print(len(df_ratings['user_id'].unique()))

In [ ]:
user_rating_count = df_ratings.groupby('user_id').size().sort_values(ascending=False)
user_rating_count

In [ ]:
book_rating_count = df_ratings.groupby('title').size().sort_values(ascending=False)
book_rating_count

In [ ]:
while True:
    book_rating_count = df_ratings['title'].value_counts()
    filtered_books = book_rating_count[book_rating_count > 20].index
    user_rating_count = df_ratings[df_ratings['title'].isin(filtered_books)]['user_id'].value_counts()
    filtered_users = user_rating_count[user_rating_count > 20].index
    df_ratings_filtered = df_ratings[
        df_ratings['title'].isin(filtered_books) &
        df_ratings['user_id'].isin(filtered_users)
    ]
    if len(df_ratings_filtered) == len(df_ratings):
        break


    df_ratings = df_ratings_filtered.copy()
df_ratings_filtered.reset_index(drop=True, inplace=True)
print(df_ratings_filtered)

In [ ]:
user_rating_count = df_ratings.groupby('user_id').size().sort_values(ascending=False)
user_rating_count

In [ ]:
book_rating_count = df_ratings.groupby('title').size().sort_values(ascending=False)
book_rating_count

In [ ]:
sns.histplot(df_ratings['rating'], bins = 5)

In [ ]:
book_index = {book: idx for idx, book in enumerate(df_ratings['title'].unique())}
book_index.get('Pride and Prejudice')

In [ ]:
user_index = {user: idx for idx, user in enumerate(df_ratings['user_id'].unique())}
user_index.get('A30TK6U7DNS82R')

In [ ]:
df_ratings.loc[:, 'book_index'] = df_ratings['title'].map(book_index)
df_ratings.loc[:, 'user_index'] = df_ratings['user_id'].map(user_index)

df_ratings.to_csv('/kaggle/working/processed_ratings.csv', index=False)

# save books detail
filtered_books = df_books[df_books['title'].isin(df_ratings['title'].unique())]
filtered_books = filtered_books.merge(
    df_ratings[['title', 'book_index']].drop_duplicates(), 
    on='title',
    how='left'
)
filtered_books.to_csv('/kaggle/working/books.csv', index=False)


print(df_ratings)

In [ ]:
book_pivot = df_ratings.pivot_table(columns='user_index', index='book_index', values="rating")
book_pivot.head(100)

In [ ]:
book_pivot.fillna(0,inplace=True)
book_pivot.head()

In [ ]:
from scipy.sparse import csr_matrix
book_sparse = csr_matrix(book_pivot)

In [ ]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute')
model.fit(book_sparse)

In [ ]:
distances, suggestions = model.kneighbors(book_pivot.iloc[237, :].values.reshape(1, -1), n_neighbors=6)
distances

In [ ]:
suggestions

In [ ]:
for i in range(len(suggestions)):
  print(book_pivot.index[suggestions[i]])

In [ ]:
book_pivot.index[3]